##### Copyright 2020 The Cirq Developers

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get started with Quantum Computing Service

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/tutorials/google/start"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/tutorials/google/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/tutorials/google/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/tutorials/google/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [2]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")

Quantum Computing Service enables researchers to run quantum programs on Google's quantum processors. This notebook is a tutorial to get you started with the typical setup, using the open source Python framework [Cirq](https://github.com/quantumlib/Cirq), in the free cloud Jupyter notebook environment, [Google Colab](https://colab.research.google.com/).

Access is currently restricted to those in an approved group, and you must be in that group before running this tutorial.

You can find more about running this in colaboratory in the [Colab documentation](https://colab.research.google.com/notebooks/welcome.ipynb) or in our Cirq-specific guide to [running in Colab](colab.ipynb).  You can download this notebook from the [GitHub repository](https://github.com/quantumlib/Cirq/blob/master/docs/tutorials/google/start.ipynb).

## Before you begin

*  First, decide which project you will use the Quantum Computing Services from.  All of your quantum programs and results will live under a project which you specify when creating and running these programs using Quantum Engine.  You can use an existing project or create a new project.  [Learn more about creating a project](https://cloud.google.com/docs/overview/).
*  Log in and agree to [Terms of Service](https://cloud.google.com/terms/).
*  Follow this link to [**enable the Quantum Engine API**](https://console.cloud.google.com/apis/library/quantum.googleapis.com?returnUrl=quantum) in your Google Cloud Platform project.

After the API is enabled, you should be redirected to the [Quantum Engine console](https://console.cloud.google.com/quantum) and it should look like the following screenshot.

![Quantum Engine landing](https://raw.githubusercontent.com/quantumlib/Cirq/master/docs/images/console-landing.png)

**Enter your project id into the input text box below**. To find your project id, click on the project menu in the blue bar at the top of the console. This will open a menu that displays your project name (e.g. "My project") and unique **project id** (e.g. my-project-1234). Enter the **project id** into the input below. ([Help on finding your project id](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects).)

![Find project id](https://raw.githubusercontent.com/quantumlib/Cirq/master/docs/images/console-project-id.png)

**Run the code in the next block (the one with the text box)**, which will prompt you to authenticate Google Cloud SDK to use your project.  You can run the code by either clicking the play button (pointed by arrow below) or by selecting the block and pressing CTRL-ENTER.  After running the block you will see a link which you should click.  This will open a new browser window.  Follow the authentication flow for this window.  After you authenticate and allow access to your project, you will be given a string which you should enter into the text box that appears in the run area (and then press return).  If you see "Authentication complete" you have done this step successfully. If this fails, make sure that you have cut and paste the string correctly (e.g. the clipboard button seems to not work for some browser/OS combinations). 

![Run code block below](https://raw.githubusercontent.com/quantumlib/Cirq/master/docs/images/run-code-block.png)

In [3]:
# The Google Cloud Project id to use.
project_id = '' #@param {type:"string"}

if project_id == '':
    import os 
    if 'GOOGLE_CLOUD_PROJECT' not in os.environ:
        raise Exception("Please setup project_id in this cell or set the `GOOGLE_CLOUD_PROJECT` env var to your project id.")
    project_id = os.environ['GOOGLE_CLOUD_PROJECT']
else: 
    import os
    os.environ['GOOGLE_CLOUD_PROJECT'] = project_id

def authenticate_user():
  """Runs the user through the Colab OAuth process.
  
  Checks for Google Application Default Credentials and runs interactive login 
  if the notebook is executed in Colab. In case the notebook is executed in Jupyter notebook
  or other IPython runtimes, no interactive login is provided, it is assumed that the 
  `GOOGLE_APPLICATION_CREDENTIALS` env var is set or `gcloud auth application-default login`
  was executed already.
  
  For more information on using Application Default Credentials see 
  https://cloud.google.com/docs/authentication/production
  """
  in_colab = False
  try:
     from IPython import get_ipython
     in_colab = 'google.colab' in str(get_ipython())
  except: 
     # Notebook is not executed within IPython. Assuming external authentication.
     return 

  if in_colab: 
      from google.colab import auth      
      print("Getting OAuth2 credentials.")
      print("Press enter after entering the verification code.")
      auth.authenticate_user(clear_output=False)
      print("Authentication complete.")
  else: 
      print("Notebook is not executed with Colab, assuming Application Default Credentials are setup.") 

authenticate_user()

print("Successful authentication to Google Cloud.")

Notebook is not executed with Colab, assuming Application Default Credentials are setup.
Successful authentication to Google Cloud.


**Authentication details** Double clicking on the project_id block above should expose the code that is run when you run this code block.  This code uses the [colabtools](https://github.com/googlecolab/colabtools/blob/master/google/colab/auth.py) auth module to ensure that *Application Default Credentials* are set and then creates a variable `colab_auth` which can be used in Cirq to authenticate your calls to Quantum Computing Service.

If you are going to run code outside of colab and want to authenticate, see the below section on running from the command-line.

## Create a circuit
Now that you've enabled Quantum Computing Service and configured the notebook, let's create a basic program with Cirq. After reviewing the code, **run this block** to run a circuit, and print a circuit diagram and results. To learn more, refer to the [Intro to Cirq](../basics.ipynb) and [Cirq documentation](https://cirq.readthedocs.io).

In [4]:
import cirq
import cirq.google as cg
import random
import string

# Define a qubit at an arbitrary grid location.
qubit = cirq.GridQubit(0, 0)

# Create a circuit (qubits start in the |0> state).
circuit = cirq.Circuit(
    cirq.X(qubit),                     # NOT gate.
    cirq.measure(qubit, key='result')  # Measurement.
)

print("Circuit:")
print(circuit)

Circuit:
(0, 0): ───X───M('result')───


## Simulate the circuit using Cirq

Let's quickly use Cirq to simulate the circuit above.

In [5]:
# Simulate the circuit, repeating 1000 times.
print("Simulating circuit using Cirq...\n")
results = cirq.sample(circuit, repetitions=1000)
print("Measurement results:")
print(results)

Simulating circuit using Cirq...

Measurement results:
result=11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

## Run on quantum hardware
Approved users can access quantum hardware in two modes. First, all approved users have access to a processor in "open-swim" which is a first-in-first-out queue with fairness algorithm that balances jobs across users in the queue. Secondly, processors can be reserved in hourly blocks if the user is approved.  You can learn more about the reservation system on the [concepts page](../../google/engine.md). We'll use the processor `pacific` in this demo.

### Create a Quantum Engine client
Interactions with hardware are facilitated by the Quantum Computing Service. A client must first be initialized with your Google Cloud project to perform these interactions. 

In [6]:
# Create an Engine client.
engine = cirq.google.get_engine()

### View the processor's topology
Each processor has a set of available qubits laid out on a grid with limited couplings between qubits. The device specification can be printed to inspect the topology of a processor.

In [ ]:
processor = engine.get_processor('pacific')

# Print the device showing qubit connectivity for the Sycamore gateset.
device = processor.get_device([cg.SYC_GATESET])
print(device)

                                             (0, 5)───(0, 6)
                                             │        │
                                             │        │
                                             (1, 5)───(1, 6)───(1, 7)
                                             │        │        │
                                             │        │        │
                                    (2, 4)───(2, 5)───(2, 6)───(2, 7)───(2, 8)
                                             │        │        │        │
                                             │        │        │        │
                                             (3, 5)───(3, 6)───(3, 7)───(3, 8)───(3, 9)
                                             │        │        │
                                             │        │        │
         (4, 1)───(4, 2)            (4, 4)───(4, 5)───(4, 6)───(4, 7)
         │        │                 │        │        │        │
         │        │                 │        │ 

Note that the qubit that we used for the simulation above, `(0, 0)`, does not exist on the hardware. Since the grid may change over time, we'll programatically select a valid qubit by inspecting `device.qubits`. We then use the `optimized_for_sycamore()` function to remap the circuit description for our device; this function will also substitute gates to comply with the device's gate set.

In [7]:
valid_qubit = device.qubits[0]

# Re-map qubits and gates for Sycamore hardware.
syc_circuit = cg.optimized_for_sycamore(
    circuit,
    new_device=device,
    qubit_map=lambda q: valid_qubit) # Constant fn since there's only one qubit.

print(syc_circuit)

(5, 2): ───PhXZ(a=-1,x=1,z=0)───M('result')───


### Create a job on the Quantum Engine

Cirq circuits are represented in the Quantum Computing Service as _Programs_. To run a _Program_, you must create a _Job_ that specifies details about the execution, _e.g._ the processor to use and the number of times to repeat the experiment. This enables a single circuit to be run multiple times in different configurations. For a one-off use, these steps can be done together by using the `engine.run_sweep` utility to create both a _Program_ and a _Job_.

A new _Job_ will be scheduled on the requested hardware as it becomes available. The execution of your _Job_ will likely be completed within a few seconds and the results will be displayed below. The output will include a link to the [console](https://console.cloud.google.com/quantum), where you can view the status and results of your jobs.

In [8]:
print("Uploading program and scheduling job on the Quantum Engine...\n")

# Upload the program and submit jobs to run in one call.
job = engine.run_sweep(
    program=syc_circuit,
    repetitions=10000,
    processor_ids=[processor.processor_id],
    gate_set=cg.SYC_GATESET)

print("Scheduled. View the job at: https://console.cloud.google.com/quantum/"
      "programs/{}?&project={}".format(job.program_id, project_id))

# Print out the results. This blocks until the results are returned.
results = [str(int(b)) for b in job.results()[0].measurements['result'][:, 0]]
print("Measurement results:\n")
print(''.join(results))

Uploading program and scheduling job on the Quantum Engine...

Scheduled. View the job at: https://console.cloud.google.com/quantum/programs/prog-UEPDBU3CSSULX8IW200829?&project=quantum-cloud-client
Measurement results:

11111111111111111011111111111111111101111011111111111111111111111111111111111111111111111111111111111111111111111111111111011011111111111111111111111111111111111111111111111111101111111111111111111111111111111111111111101111111111111111111111111111111111111111111111111011111111111111111111111101110111111111111111111111111111111111111111111111111111111111110111111111111111111111111111111111111111011111111111111111111111111111111111111111111101111111111011111111111111111111111111111111111111011111011111111111111110111111111111111111110111111111111111111111111101111011111111111111101111111111111111111111111111011111111111111111111111111111111111111111111111111111111111111111111111111111111110111111111111011111111111111011111111111110111111011111111111101111110111111111111

## Running from the command line

If you are planning to access Quantum Computing Service from the command line, follow these instructions to get started.  If you plan on executing all of your programs from an ipython notebook, you can skip this section.

We assume that your local machine is configured to run Python 3.6 or greater.  

Instructions for installing Python 3:

*   Linux: https://docs.python-guide.org/starting/install3/linux/
*   Mac: https://docs.python-guide.org/starting/install3/osx/
*   Windows:  If you are using Windows Subsystem for Linux: https://docs.python-guide.org/starting/install3/linux/, otherwise: https://docs.python-guide.org/starting/install3/win/

### Setup authentication

In this quickstart we will authenticate using the gcloud command line cool.  To do this, one must first install gcloud.  Follow the instructions for this at https://cloud.google.com/sdk/install.  If you are running on a corp desktop, gcloud should be installed.  We will authenticate using Application Default Credentials.  To do this simply run the following on your shell command line

`gcloud auth application-default login`

This will open up a browser window or give you a link to a webpage you can navigate to in order to go through an authentication flow.  Complete this using your @google.com account.  After this command is run, credentials will be stored on your local machine.  If at any point you want to revoke these credentials you can run `gcloud auth application-default revoke`

We highly recommend that you setup a virtual environment for this installation to isolate your development stack from your overall system installations.  Instructions for virtual environments are at https://packaging.python.org/guides/installing-using-pip-and-virtual-environments/.  Make sure to setup the virtual environment for Python 3 and not Python 2.

### Write and run a short quantum program

Using your favorite IDE or editor, read and then paste the following hello_qubit program into a file called `hello_qubit.py`.  Make sure to replace the `'your-project-id'` string with the project id you created above.


In [9]:
import cirq

def example():
    """Hello qubit example run against a quantum processor."""

    # Define a qubit.
    qubit = cirq.GridQubit(5, 2)

    # Create a circuit (qubits start in the |0> state).
    circuit = cirq.Circuit(
        cirq.X(qubit)**0.5,                # Square root of NOT.
        cirq.measure(qubit, key='result')  # Measurement.
    )

    # Create an Engine object.  This uses the project id of your
    # Google cloud project.
    project_id = 'your-project-id'
    engine = cirq.google.Engine(project_id=project_id)

    print("Uploading program and scheduling job on Quantum Engine...\n")

    # Upload the program and submit jobs to run in one call.
    job = engine.run_sweep(
        program=circuit,
        repetitions=1000,
        processor_ids=['pacific'],
        gate_set=cirq.google.SYC_GATESET)

    print("Scheduled. View the job at: https://console.cloud.google.com/quantum/"
          f"programs/{job.program_id}/jobs/{job.job_id}"
          f"/overview?project={project_id}")

    # Print out the results. This blocks until the results are returned.
    results = [str(int(b)) for b in job.results()[0].measurements['result'][:, 0]]
    print("\nMeasurement results:")
    print(''.join(results))


if __name__ == '__main__':
    example()

You should then be able to run this program from your command line using:

`python hello_qubit.py`

The output should be something like:

```
Uploading program and scheduling job on Quantum Engine...

Scheduled. View the job at: https://console.cloud.google.com/quantum/programs/example-T5K9Y9/jobs/job-0?mods=quantum_ng2&project=quantum-cloud-client

Measurement results:
1010101010111011000011101010101011010011100111110001101101101011100011111010100100011000010101110010011000100001110000100010011100111101001101101010000000111101010100001001101110101010110000010011110101100000101111101000010000000100010111010101110001110101101100010001101111100110001001010000100000110110100000111111110011100101111000010110010101100010111111100010001100111011010101111011000110100001010000011100000100101010111110111010011010100011011111110110011101010001000101000011101000111000001001101000110000111111000111100001001000010101011111000111111010100001011001010011011000010110111010001011000001011110001010110100001101011101110011111100100101010011100010010001100101100001101010010011001010100011010010011000010010001010111101001000011000101111011111100111111111101001010010001010000101011101100100001101011100101010101110111001101000100101111101100000100010000101000011110110101001101100000000000111000000111011101101110000001110110001111011000100111111101110111111101001110110000110
```


## Next steps
* Use [this template colab](colab.ipynb) as a base for your own explorations.
* Explore [best practices](../../google/best_practices.md) for getting circuits to run on hardware.
* Use the [reservation colab](reservations.ipynb) for getting circuits to run on hardware.